# CV Project Jupyter Script

## Clone Project From ( Optional )

In [ ]:
PullFromRepo = False

if PullFromRepo:
    !git init
    !git remote add origin "https://github.com/freshLiver/CV_PROJECT"
    !git pull origin test
    pass


## Imports and Hyper-parameters

### Imports

In [ ]:
import torch
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from light_cnn import LightCNN_9Layers
from functional import TrainingHelper, ImageList


### Hyper-parameters

In [ ]:
# training
EPOCHS = 5
BS = 128             # batch size
LR = 0.0001          # learning rate
NUM_WORKER = 2
PRINT_FREQUENCY = 10
VALID_RATIO = 0.2


## Data Pre-processing

### Download and Unzip Dataset

In [ ]:
import math
import random
from os import system
from pathlib import Path

# give me abs path
ROOT = Path("/tmp")
TRAIN_LIST = Path.joinpath(ROOT, "train_list.txt")
VALID_LIST = Path.joinpath(ROOT, "valid_list.txt")

# download dataset
system(f"wget -cP {ROOT} http://vis-www.cs.umass.edu/lfw/lfw.tgz")

# extract if dir not exists
dataset_dir = Path(f'{ROOT}/lfw')
if not dataset_dir.exists():
    system(f"tar -xf {ROOT}/lfw.tgz --directory {ROOT}")


### Parse Dataset

In [ ]:
# load data into dict
dataset = []

NUM_CLASSES = 0
for index, catdir in enumerate(dataset_dir.glob("*")):
    # iterate each file in this dir
    for pth in catdir.glob("*"):
        dataset.append((pth, index))
    NUM_CLASSES += 1

    # TODO trick
    if NUM_CLASSES == 100:
        break

random.shuffle(dataset)

# split dataset
train_size = math.floor(len(dataset) * (1-VALID_RATIO))
valid_size = len(dataset) - train_size


### Save Dataset (Train Dataset, Valid Dataset)

In [ ]:
# save to file
with open(TRAIN_LIST, 'w') as f:
    for img, cat in dataset[:train_size]:
        f.write(f'{img} {cat}\n')

with open(VALID_LIST, 'w') as f:
    for img, cat in dataset[train_size:]:
        f.write(f'{img} {cat}\n')


## Main Script

### Choose Model

In [ ]:
from torchvision.models.vgg import vgg16

model = vgg16(pretrained=False, num_classes=NUM_CLASSES)

# large lr for last fc parameters
if torch.cuda.is_available():
    model = model.cuda()

print(model)


### DataLoader

In [ ]:
# define transforms
train_transform = transforms.Compose([
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

valid_transform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])


# create data loaders
train_loader = DataLoader(
    ImageList(root=ROOT, fileList=TRAIN_LIST, transform=train_transform),
    batch_size=BS,
    shuffle=True,
    num_workers=NUM_WORKER,
    pin_memory=True
)

valid_loader = DataLoader(
    ImageList(root=ROOT, fileList=VALID_LIST, transform=valid_transform),
    batch_size=BS,
    shuffle=False,
    num_workers=NUM_WORKER,
    pin_memory=True
)


### Train !

In [ ]:
helper = TrainingHelper(
    train_dataloader=train_loader,
    valid_dataloader=valid_loader,
    epochs=EPOCHS,
    batch_size=BS,
    learning_rate=LR,
    print_frequency=PRINT_FREQUENCY,
    model=model,
    criterion=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(params=model.parameters(), lr=LR)
)

cudnn.benchmark = True

helper.validate()
for epoch in range(0, EPOCHS):

    helper.adjust_learning_rate(epoch)

    # train for one epoch
    helper.train(epoch)

    # evaluate on validation set
    prec1 = helper.validate()
